抓取果壳网列表页的内容，报错标题，url地址，简介，点赞数等：https://www.guokr.com/ask/highlight/


In [ ]:
import scrapy
import urllib.parse


class GkSpider(scrapy.Spider):
    name = 'gk'
    allowed_domains = ['guokr.com']
    start_urls = ['https://www.guokr.com/ask/highlight/']

    def parse(self, response):

        # 进行数据的提取
        # 先分组，再提取
        li_list = response.xpath("//ul[@class='ask-list-cp']/li")
        for li in li_list:
            item = {}
            # 关注数
            item["focus_nums"] = li.xpath(
                ".//p[@class='ask-focus-nums']/span/text()").extract_first()
            # 回答数
            item["answer_nums"] = li.xpath(
                ".//p[@class='ask-answer-nums']/span/text()").extract_first()
            # url
            item["href"] = li.xpath(".//h2/a/@href").extract_first()
            # 标题
            item["title"] = li.xpath(".//h2/a/text()").extract_first()
            # 描述
            item["summary"] = li.xpath(
                ".//p[@class='ask-list-summary']/text()").extract_first().strip()
            # 标签
            item["tag"] = li.xpath(".//a[@class='tag']/text()").extract()
            print(item)

        # 寻找下一页的url地址，构造请求，交给引擎，指定callback函数
        next_url = response.xpath("//a[text()='下一页']/@href").extract_first()
        if next_url is not None:
            # next_url = urllib.parse.urljoin(response.url,next_url)
            print("*"*100)
            # yield scrapy.Request(
            #     next_url,
            #     callback=self.parse  #用自己来提取下一页的数据
            # )

            # 使用response.follow方法，发送请求
            yield response.follow(
                next_url,  # 不完整的url地址，根据response的url拼接完整之后构造request对象
                callback=self.parse
            )
